In [1]:
import os
import re
import google
from google.oauth2 import credentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
import pandas as pd 
import numpy as np
from google.cloud import bigquery
from google.cloud import storage
import gc
import time
from datetime import datetime

#`divg-josh-pr-d1cc3a.campaign_performance_analysis.contract_expiry_and_discounts` 

project_id = 'divg-josh-pr-d1cc3a' 
dataset_id = 'promo_expiry_analysis'
tbl_nm = 'latest_bill_discount_apr_2023_cohort'
save_data_path='gs://divg-josh-pr-d1cc3a-default/downloads/{}.csv'.format(tbl_nm)

def get_gcp_bqclient(project_id, use_local_credential=True):
    token = os.popen('gcloud auth print-access-token').read()
    token = re.sub(f'\n$', '', token)
    credentials = google.oauth2.credentials.Credentials(token)

    bq_client = bigquery.Client(project=project_id)
    if use_local_credential:
        bq_client = bigquery.Client(project=project_id, credentials=credentials)
    return bq_client

client = get_gcp_bqclient(project_id)

data_set = f"{project_id}.{dataset_id}.{tbl_nm}" 

sql = '''SELECT * FROM `{data_set}`'''.format(data_set=data_set)

df_export = client.query(sql).to_dataframe()

df_export.head()

# df_export = df_export.reset_index()
# df_export.to_csv(save_data_path, index=False) 

# print(f"{data_set} saved in {save_data_path}")


,BAN,latest_bill_dt,bill_doc_id,bill_tot_inv_amt,bill_tot_tax_amt,bill_chrg_dtl_id,bill_chrg_dtl_typ_cd,chrg_rev_cd,srvc_resrc_typ_cd,bill_itm_dsply_nm,discount_start_date,discount_end_date,chrg_typ_cd,net_chrg_amt
0,237095542,2023-02-04,1251155603.000000000,56.880000000,5.520000000,46282840328.000000000,DI,RC,HSIC,24 MO - Unlimited Internet Usage Discount,2023-02-04 00:00:00,2023-03-03 00:00:00,DSC,-20.000000000
1,237095542,2023-02-04,1251155603.000000000,56.880000000,5.520000000,46282840318.000000000,RC,RC,HSIC,Digital Box Service Agreement Rental Discount,9999-12-31 00:00:00,9999-12-31 00:00:00,DSC,-5.000000000
2,237095542,2023-02-04,1251155603.000000000,56.880000000,5.520000000,46282840354.000000000,RC,RC,HSIC,24 MO - Internet Loyalty Credit - $10 incl tax...,9999-12-31 00:00:00,9999-12-31 00:00:00,DSC,-8.930000000
3,600992819,2023-02-21,1245262295.000000000,8.140000000,1.680000000,46113526567.000000000,RC,RC,TTV,"12 MO - Optik TV Discount - effective Jun 22, ...",9999-12-31 00:00:00,9999-12-31 00:00:00,DSC,-25.000000000
4,600992819,2023-02-21,1245262295.000000000,8.140000000,1.680000000,46113526666.000000000,RC,RC,SMHM,Promotional Discount - 36 month - effective Ju...,9999-12-31 00:00:00,9999-12-31 00:00:00,DSC,-7.000000000


In [2]:
bill_itm_dsply_nm = df_export['bill_itm_dsply_nm'].to_list() 
discount_end_date = df_export['discount_end_date'].to_list() 
discount_end_dt = []

def left(s, amount):
    return s[:amount]

for idx, item in enumerate(bill_itm_dsply_nm): 
    if 'discount end' in item and re.search(r'[a-zA-Z]{3} [0-9]{2}, [0-9]{4}', item): 
        discount_end_dt.append(datetime.strptime(re.search(r'[a-zA-Z]{3} [0-9]{2}, [0-9]{4}', item)[0], "%b %d, %Y").date())
    elif re.search(r'effective [a-zA-Z]{3} [0-9]{2}, [0-9]{4} to [a-zA-Z]{3} [0-9]{2}, [0-9]{4}', item): 
        discount_end_dt.append(datetime.strptime(re.findall(r'[a-zA-Z]{3} [0-9]{2}, [0-9]{4}', item)[1], "%b %d, %Y").date())
    else: 
        discount_end_dt.append(datetime.strptime(left(str(discount_end_date[idx]), 10), "%Y-%m-%d").date())
    
df_discount_end_dt = pd.DataFrame(discount_end_dt, columns=['df_discount_end_dt'])       

df_export['df_discount_end_dt'] = df_discount_end_dt

df_export = df_export.reset_index()
df_export.to_csv(save_data_path, index=False) 

print(f"{data_set} saved in {save_data_path}")


divg-josh-pr-d1cc3a.promo_expiry_analysis.latest_bill_discount_apr_2023_cohort saved in gs://divg-josh-pr-d1cc3a-default/downloads/latest_bill_discount_apr_2023_cohort.csv


In [3]:
from datetime import datetime

dt = re.search(r'[a-zA-Z]{3} [0-9]{2}, [0-9]{4}', df_export['bill_itm_dsply_nm'].to_list()[-5000])[0]

print(datetime.strptime(dt, "%b %d, %Y").date()) 


2021-05-22
